<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-10-14 10:32:58--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-10-14 10:32:59--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  30.6MB/s    in 53s     

2024-10-14 10:33:52 (26.1 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [5]:
# method most_simmilar
wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

## входит, 4-е место

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
#class MyTokenizer:
#    def __init__(self):
#        pass
#    def tokenize(self, text):
#        return re.findall('\w+', text)
tokenizer = WordPunctTokenizer()

In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question.lower())
    question_emb = np.zeros(dim)
    divider = 0
    for token in tokens:
      if token in embeddings:
        question_emb += embeddings[token]
        divider += 1
    return np.zeros(200) if divider == 0 else question_emb/divider

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [8]:
question_to_vec('I love neural networks', wv_embeddings, tokenizer)[2]

-1.2854121724764507

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [9]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    score = 0
    for rank in dup_ranks:
      if rank <= k:
        score += 1
    return score

In [10]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    score = 0
    for rank in dup_ranks:
        if rank <= k:
          score += 1/math.log2(rank+1)
    return score

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [11]:
import pandas as pd

In [12]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]
x = question_to_vec(copy_answers[0], wv_embeddings, tokenizer)
None

In [13]:
'dog' in wv_embeddings

True

In [14]:
from scipy.spatial.distance import cosine

def cosine_similarity_(vec1: np.ndarray, vec2:np.ndarray) -> float:
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1

ans_emb = question_to_vec(copy_answers[0], wv_embeddings, tokenizer)
candidat_embs = [question_to_vec(candidates_ranking[0][k], wv_embeddings, tokenizer) for k in range(len(candidates_ranking[0]))]
cosine_dists = [cosine_similarity_(ans_emb, candidat_embs[k]) for k in range(len(candidat_embs))]

print(cosine_dists)
dup_ranks = [2] # я не понял почему надо 2 передавать для правильного ответа, вообще не понимаю
# во первых я думал что отсчет начинается с 0 а не с 1, во вторых candidates_ranking не является ранжированным списком


# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

[-0.0006016496821679156, 0.9999999999999999, 0.37646355421981637, -0.0828166767624061]
Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [15]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [16]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [50]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
      list_of_q = line.strip().lower().split('\t')

      data.append(list_of_q)
    return data

Нам понадобиться только файл validation.

In [51]:
validation_data = read_corpus('validation.tsv')

Кол-во строк

In [52]:
len(validation_data)

3760

Размер нескольких первых строк

In [53]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [72]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    q_emb = question_to_vec(question, embeddings, tokenizer)
    cand_embs = [question_to_vec(candidates[k], embeddings, tokenizer) for k in range(len(candidates))]

    cosine_dists = []
    for i in range(len(cand_embs)):
      cos_sim = cosine_similarity_(q_emb, cand_embs[i])
      cosine_dists.append(cos_sim)

    sorted_cosine_dists = sorted(cosine_dists, reverse=True)
    ranks = []
    for i in range(len(sorted_cosine_dists)):
      curr_emb_rank = sorted_cosine_dists[i]
      curr_emb_prev_index = cosine_dists.index(curr_emb_rank)
      ranks.append((curr_emb_prev_index, candidates[curr_emb_prev_index]))
    return ranks

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [73]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [74]:
for question, q_candidates in zip(questions, candidates):

  ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
  print(ranks)
  print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [75]:
from tqdm.notebook import tqdm

In [88]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [89]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 410.000 | Hits@   1: 410.000
DCG@   5: 499.894 | Hits@   5: 580.000
DCG@  10: 520.969 | Hits@  10: 645.000
DCG@ 100: 567.579 | Hits@ 100: 875.000
DCG@ 500: 580.516 | Hits@ 500: 973.000
DCG@1000: 583.415 | Hits@1000: 1000.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [93]:
train_data = read_corpus('train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

В задачах, связанных с пониманием контекста (как здесь — схожие вопросы), разумно использовать размер окна от 5 до 10. Если окно слишком маленькое, модель не будет улавливать связь между словами на дальнем расстоянии; слишком большое окно может "размыть" контекст.

In [101]:
words = [question[0].split() + question[1].split() for question in train_data]

In [103]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [105]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

<ipython-input-14-1887b6b8d31f>:4: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


In [106]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 294.000 | Hits@   1: 294.000
DCG@   5: 367.097 | Hits@   5: 432.000
DCG@  10: 389.623 | Hits@  10: 501.000
DCG@ 100: 442.691 | Hits@ 100: 763.000
DCG@ 500: 465.116 | Hits@ 500: 937.000
DCG@1000: 471.835 | Hits@1000: 1000.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1) я использовал только WordPunkTokenizer, по этому не могу сказать за другие типы токенизации, однако более сложная токенизация, которая учитывает пунктуацию, регистр и многословные выражения, может дать лучшее качество, так как такие токены лучше соответствуют контексту, а ключевые фразы будут захвачены более корректно. Для такой задачи простой токенизации достаточно, но если заниматься более серьезными задачами NLP то нужны соответствующие способы токенизации, например ВРЕ.

2) В данном случае нормализация слов помогает получить более согласованные и компактные векторы для похожих слов. Насколько я понимаю, к нормализации относят такие преобразования как приведение к нижнему регистру (это очень важно, потому что слова "Окно" и "окно" для программы разные), лемматизация (очень важна для русского где много падежей и сущесвуют рода, что увеличивает разнообразие одного и того же слова), стемминг (также полезен, но может приводить к чрезмерной агрессивности при усечении слов, что иногда ухудшает результаты).

3) Word2Vec, будучи одной из популярных моделей для создания эмбеддингов, хорошо справляется с семантическими связями между словами. Однако для более сложных задач, таких как поиск семантически похожих вопросов, более мощные модели, такие как BERT, RoBERTa или SBERT (Sentence-BERT), могут показать лучшие результаты.

4) простая токенизация, маленький объем данных и использование word2vec в качестве эмбеддера

5) можно попробовать расширить датасет, использовать ВРЕ токенизацию обучившись на пуле вопросов из форума (чем больше тем лучше) и в конце концов обучить нормально эмбеддинги, использовать нейронную сеть для этого и для решения этой задачи, а не просто косинусное расстояние. Также неплохо с этой задачей (ранжирование) справляется Td-idf